Clase 8
======

Anuncios
--------------

Por favor llenen la siguiente [encuesta](https://docs.google.com/forms/d/e/1FAIpQLSd0DyU1DYb2y2KBshDuzOxrAf-f459NRsu9_lIwgyo83FmD5w/viewform?usp=sf_link).


Introducción
------------------


En la clase pasado vimos como rutas alternativas y ciclos resultan en más de una configuración de flujos siendo compatible con los flujos de entrada y salida de un red metabólica:

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase8_alternativasCiclos.png" width="400">

Hoy veremos dos herramientas para abordar esta ambiguedad en la solución de FBA:

* Flux variability análisis
* Flux sampling

Flux variability análisis
--------

FVA (flux variability análisis) encuentra el rango de cada flujo metabólico en el valor óptimo.

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png" width="400">

In [1]:
from cobra import Model, Reaction, Metabolite
cobra_model = Model('primordialMetabolism')
# Ractions
E1 = Reaction('E1')
E1.name = "Exchange reaction of CO2"
E1.subsystem = 'Entrance'
E1.lower_bound = 0.  
E1.upper_bound = 10.  
E2 = Reaction('E2')
E2.name = 'Exchange reaction of H2'
E2.subsystem = 'Entrance'
E2.lower_bound = 0.  
E2.upper_bound = 1000.  
R1 = Reaction('R1')
R1.name = 'First reaction'
R1.subsystem = 'Internal reaction'
R1.lower_bound = 0.  
R1.upper_bound = 1000. 
R2 = Reaction('R2')
R2.name = 'Second reaction'
R2.subsystem = 'Internal reaction'
R2.lower_bound = 0.  
R2.upper_bound = 1000.  
R3 = Reaction('R3')
R3.name = 'Third reaction'
R3.subsystem = 'Internal reaction'
R3.lower_bound = 0.  
R3.upper_bound = 1000. 
E3 = Reaction('E3')
E3.name = 'Exchange reaction of CH4'
E3.subsystem = 'Exit'
E3.lower_bound = 0.  
E3.upper_bound = 1000.  
E4 = Reaction('E4')
E4.name = 'Exchange reaction of F420'
E4.subsystem = 'Exit'
E4.lower_bound = 0. 
E4.upper_bound = 1000. 
E5 = Reaction('E5')
E5.name = 'Exchange reaction of H20'
E5.subsystem = 'Exit'
E5.lower_bound = 0.  
E5.upper_bound = 1000. 
# Metabolites
CO2 = Metabolite('CO2', formula='CO2',name='carbon dioxide',compartment='c')
H2 = Metabolite('H2', formula='H2',name='H2',compartment='c')
F420 = Metabolite('F420', formula='F420',name='F420',compartment='c')
I1 = Metabolite('I1', formula='I1',name='I1',compartment='c')
H2O = Metabolite('H2O', formula='H2O',name='H2O',compartment='c')
CH4 = Metabolite('CH4', formula='CH4',name='CH4',compartment='c')
# Adding metabolites to reactions
E1.add_metabolites({CO2: 1.0})
E2.add_metabolites({H2: 1.0})
E3.add_metabolites({CH4: -1.0})
E4.add_metabolites({F420: -2})
E5.add_metabolites({H2O: -1})
R1.add_metabolites({CO2:-1,H2: -1.0,F420:-1,I1:1})
R2.add_metabolites({H2:-1,F420:1,I1:-1,CH4:1})
R3.add_metabolites({F420:1,I1:1,CH4:-1,H2O:1})
# Adding reactions to model 
cobra_model.add_reaction(E1)
cobra_model.add_reaction(E2)
cobra_model.add_reaction(R1)
cobra_model.add_reaction(R2)
cobra_model.add_reaction(R3)
cobra_model.add_reaction(E3)
cobra_model.add_reaction(E4)
cobra_model.add_reaction(E5)
# Define objective function
cobra_model.objective = 'E3'
solution = cobra_model.optimize()

In [2]:
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(cobra_model, cobra_model.reactions[:8])

,minimum,maximum
E1,10.0,10.0
E2,20.0,1000.0
R1,10.0,10.0
R2,10.0,990.0
R3,0.0,980.0
E3,10.0,10.0
E4,0.0,980.0
E5,0.0,980.0


Flux sampling
-------

Otra alternativa para explorar el espacio de soluciones alternativas de FBA es tomar muestrasa aleatorias. Esto se llama Flux sampling.

In [8]:
from cobra.test import create_test_model
from cobra.sampling import sample
s = sample(cobra_model, 100) # tomaremos 100 muestras aleatorias
s.head(6)

,E1,E2,R1,R2,R3,E3,E4,E5
0,5.578858,703.469075,5.578858,697.890217,692.311360,5.578858,692.311360,692.311360
1,3.154628,617.856314,3.154628,614.701686,611.547059,3.154628,611.547059,611.547059
2,1.309241,58.808974,1.309241,57.499733,56.190492,1.309241,56.190492,56.190492
3,7.577828,813.808531,7.577828,806.230703,798.652875,7.577828,798.652875,798.652875
4,7.539349,370.522077,7.539349,362.982728,355.443378,7.539349,355.443378,355.443378
5,6.888347,91.041822,6.888347,84.153475,77.265128,6.888347,77.265128,77.265128


In [9]:
#Si quiero ver solo los valores de la reacción E1
s["E1"]

0     5.578858
1     3.154628
2     1.309241
3     7.577828
4     7.539349
        ...   
95    3.574605
96    4.835266
97    4.213900
98    3.012304
99    5.611660
Name: E1, Length: 100, dtype: float64

In [10]:
#Si quiero ver el valor promedio de la reacción E1 a lo largo de las 100 muestras
s["E1"].mean()

5.214572156821046

Ejercicios
==========

1. Computa FBA de la red de *Escherichia coli* usando los valores por defecto de consumo de glucos y oxigeno.
2. Realiza FVA para la solución anterior.
3. Computa 100 muestras aleatorias del espacio de soluciones alternativas del punto 1, y luego calcula el valor promedio de todas sus reacciones.
4. Grafica los resultados usando ```escher```.